In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import itertools

In [2]:
# Set random seed untuk reproduksibilitas
torch.manual_seed(42)
np.random.seed(42)

In [16]:
# **Memuat dataset**
file_path = '/content/heart.csv' # Sesuaikan path jika tidak menggunakan Colab
data = pd.read_csv(file_path)

In [17]:
# Membuat dummy data penyakit jantung
def generate_heart_disease_data(n_samples=1000):
    np.random.seed(42)

    # Fitur-fitur yang mungkin mempengaruhi penyakit jantung
    age = np.random.normal(55, 10, n_samples)
    cholesterol = np.random.normal(200, 50, n_samples)
    blood_pressure = np.random.normal(130, 20, n_samples)
    max_heart_rate = np.random.normal(150, 30, n_samples)

    # Membuat target berdasarkan kombinasi fitur
    risk_score = (
        0.3 * age +
        0.2 * cholesterol +
        0.2 * blood_pressure -
        0.1 * max_heart_rate
    )

    # Klasifikasi binary: 1 (berisiko), 0 (tidak berisiko)
    labels = (risk_score > np.median(risk_score)).astype(int)

    # Membuat DataFrame
    data = pd.DataFrame({
        'age': age,
        'cholesterol': cholesterol,
        'blood_pressure': blood_pressure,
        'max_heart_rate': max_heart_rate,
        'heart_disease': labels
    })

    return data

In [18]:
# Persiapan data
data = generate_heart_disease_data()

In [19]:
# Preprocessing
X = data.drop('heart_disease', axis=1)
y = data['heart_disease']

In [20]:
# Normalisasi data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [21]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)

In [22]:
# Konversi ke tensor PyTorch
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.FloatTensor(y_train.values).unsqueeze(1)
y_test = torch.FloatTensor(y_test.values).unsqueeze(1)

In [23]:
# Definisi Model MLP dengan berbagai konfigurasi
class HeartDiseaseClassifier(nn.Module):
    def __init__(self, input_size, hidden_layers, neurons, activation='relu'):
        super(HeartDiseaseClassifier, self).__init__()

        # Pilih activation function
        if activation == 'linear':
            act_func = nn.Identity()
        elif activation == 'sigmoid':
            act_func = nn.Sigmoid()
        elif activation == 'relu':
            act_func = nn.ReLU()
        elif activation == 'tanh':
            act_func = nn.Tanh()
        elif activation == 'softmax':
            act_func = nn.Softmax(dim=1)

        # Bangun layer-layer
        layers = []
        prev_neurons = input_size

        for _ in range(hidden_layers):
            layers.append(nn.Linear(prev_neurons, neurons))
            layers.append(act_func)
            prev_neurons = neurons

        layers.append(nn.Linear(prev_neurons, 1))
        layers.append(nn.Sigmoid())  # Output biner

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [24]:
# Fungsi Training
def train_model(model, X_train, y_train, X_test, y_test,
                epochs, learning_rate, batch_size):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True
    )

    for epoch in range(epochs):
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
# Evaluasi
    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test)
        predicted = (test_outputs > 0.5).float()
        accuracy = (predicted == y_test).float().mean()

    return accuracy.item()

In [25]:
# Grid Search Parameter
params = {
    'hidden_layers': [1, 2, 3],
    'neurons': [4, 8, 16, 32, 64],
    'activation': ['linear', 'sigmoid', 'relu', 'tanh', 'softmax'],
    'epochs': [1, 10, 25, 50, 100, 250],
    'learning_rate': [1.0, 0.1, 0.01, 0.001, 0.0001],
    'batch_size': [16, 32, 64, 128, 256, 512]
}

In [26]:
# Fungsi untuk menjalankan grid search
def grid_search(X_train, y_train, X_test, y_test):
    input_size = X_train.shape[1]
    results = []

    # Kombinasi parameter terbatas untuk efisiensi
    param_combinations = list(itertools.product(
        params['hidden_layers'][:2],  # Batasi kombinasi
        params['neurons'][:3],
        params['activation'][:3],
        params['epochs'][:3],
        params['learning_rate'][:3],
        params['batch_size'][:3]
    ))

    for config in param_combinations:
        hl, neurons, act, epochs, lr, bs = config

        model = HeartDiseaseClassifier(
            input_size,
            hidden_layers=hl,
            neurons=neurons,
            activation=act
        )

        accuracy = train_model(
            model, X_train, y_train,
            X_test, y_test,
            epochs=epochs,
            learning_rate=lr,
            batch_size=bs
        )

        results.append({
            'Hidden Layers': hl,
            'Neurons': neurons,
            'Activation': act,
            'Epochs': epochs,
            'Learning Rate': lr,
            'Batch Size': bs,
            'Accuracy': accuracy
        })

In [29]:
# Tampilkan hasil
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('Accuracy', ascending=False)
    print(results_df.head(10))

    return results_df

IndentationError: unexpected indent (<ipython-input-29-316f2442ccdb>, line 2)

In [15]:
# Jalankan Grid Search
results = grid_search(X_train, y_train, X_test, y_test)